In [1]:
import requests
from bs4 import BeautifulSoup
from datetime import datetime, timedelta
import locale
locale.setlocale(locale.LC_ALL, 'ID')
import re

In [422]:
def getIndeksLink(links, page, cat_link, category, date=datetime.strftime(datetime.today(), '%m/%d/%Y')):
    """
    Untuk mengambil seluruh url 
    link pada indeks category tertentu
    date format : dd/mm/YYYY
    """
    if cat_link == 'news':
        url = "https://"+cat_link+".detik.com/indeks/all/"+str(page)+"?date="+date
    else :
        url = "https://"+cat_link+".detik.com/indeks/"+str(page)+"?date="+date
    # Make the request and create the response object: response
    response = requests.get(url)
    # Extract HTML texts contained in Response object: html
    html = response.text
    # Create a BeautifulSoup object from the HTML: soup
    soup = BeautifulSoup(html, "html5lib")
    contentDiv = soup.find('ul', attrs={'id':'indeks-container'})
    indeks = contentDiv.findAll('article')
    for post in indeks:
        link = [post.find('a', href=True)['href'], category]
        links.append(link)
        
    el_page = soup.find('div', class_="paging paging2")
    if el_page:
        max_page = int(soup.find('div', class_="paging paging2").findAll('a')[-2].text.replace('\n', '').strip(' '))
    
        if page <= max_page:
            links = getIndeksLink(links, i+1, cat_link, category, date)
        
    return url

In [140]:
def getDetailBerita(links):
    articles = {}
    for link in links:
        #link
        articles['url'] = link[0]
        response = requests.get(url)
        html = response.text
        # Create a BeautifulSoup object from the HTML: soup
        soup = BeautifulSoup(html, "html5lib")
        
        #articleid
        articles['id'] = int(soup.find("meta", attrs={'name':'cXenseParse:articleid'})['content'])
        
        #category
        articles['category'] = link[1]
        
        #extract subcategory from breadcrumb
        bc = soup.find('ul', class_="breadcrumb__wrap")
        articles['subcategory'] = bc.findAll('li')[2].text
        
        #article
        article = soup.find("div", class_="read__content")
        
        #extract date
        pubdate = soup.find("meta", attrs={"name":"content_PublishedDate"})['content']
        pubdate = pubdate.strip(' \t\n\r')
        articles['pubdate']=datetime.strftime(datetime.strptime(pubdate, "%Y-%m-%d %H:%M:%S"), "%Y-%m-%d %H:%M:%S")
        
        #extract author
        articles['author'] = soup.find('div', class_="read__author").text
        
        #extract title
        articles['title'] = soup.find('h1', class_="read__title").text      

        #source
        articles['source'] = 'detik'
        
        #extract comments count
        #articles['comments'] = int(soup.find('a', class_="komentar").find('span').text.replace('Komentar', '').strip(' \t\n\r'))
        
        #extract tags
        tags = soup.find("meta", attrs={'name':'content_tag'})['content']
        articles['tags'] = ','.join([x.text for x in tags])
        
        #extract images
        articles['images'] = soup.find('meta', attrs={'property':'og:image'})['content']
        
       
        #hapus link sisip
#         for link in detail.findAll('table', class_="linksisip"):
#             link.decompose()
        
        #hapus video sisip
#         for tag in detail.findAll('div', class_="detail_tag"):
#             tag.decompose()
        
        #extract content
        detail = BeautifulSoup(article.decode_contents().replace('<br/>', ' '), "html5lib")
        content = re.sub(r'\n|\t|\b|\r','',detail.text)
        articles['content'] = content
        
        return articles

In [ ]:
get

In [2]:
# links = getIndeksLink([], 1, 'news', 'news')